# Facial Landmark Analysis for Detecting Visual Impairment in Mobile LogMAR Test

In [ ]:
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import re

from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, RocCurveDisplay
import xgboost as xgb
import shap

In [ ]:
plt.rcParams.update({'font.size': 12})
dpi = 300

# Study Results

In [ ]:
df = pd.read_csv('../temp/train_data.csv')

In [ ]:
df.describe()

In [ ]:
df['label_bin'] = df['label'].replace('all', 0).replace('some', 1).replace('none', 1)
df['label_mult'] = df['label'].replace('all', 0).replace('some', 1).replace('none', 2)

In [ ]:
unique_subjects_df = df.sort_values('logMAR').drop_duplicates(subset='subject', keep='first').reset_index()[['index', 'subject', 'hasVisualAid', 'usesVisualAid','needsNewVisualAid', 'visualAidDiopters', 'canReadCinemaSubtitles', 'canReadSmartphoneScreen', 'canReadRoadSigns', 'wasEasy', 'distance', 'logMAR']]

In [ ]:
unique_subjects_df['visualAidDiopters'] = unique_subjects_df['visualAidDiopters'].str.replace(',', '.').astype(float)

In [ ]:
n = len(unique_subjects_df)
hasVisualAidTrue = np.sum(unique_subjects_df.hasVisualAid)
usesVisualAidTrue = np.sum(unique_subjects_df.usesVisualAid)
needsNewVisualAidTrue = np.sum(unique_subjects_df.needsNewVisualAid)
canReadCinemaSubtitlesTrue = np.sum(unique_subjects_df.canReadCinemaSubtitles)
canReadSmartphoneScreenTrue = np.sum(unique_subjects_df.canReadSmartphoneScreen)
canReadRoadSignsTrue = np.sum(unique_subjects_df.canReadRoadSigns)
wasEasyTrue = np.sum(unique_subjects_df.wasEasy)

## TODO Values need to be reconsidered:

In [ ]:
distances = ''
q = unique_subjects_df.distance.value_counts()
for k in q.index:
    distances += str(k) + 'm: ' + str(q[k]) + ' \t '
    
print('Number of runs:\t\t', n)
print('Test conducted at \t', distances)
print(
    'Range of diopters:\t',
    np.round(np.mean(unique_subjects_df.visualAidDiopters), 2),
    '±',
    np.round(np.std(unique_subjects_df.visualAidDiopters), 2)
)
print('Do you have a visual aid?\t\t\t\t\t\t', hasVisualAidTrue, 'yes \t', n-hasVisualAidTrue, 'no')
print('Is this test being conducted with a visual aid?\t\t\t\t', usesVisualAidTrue, 'yes \t', n-usesVisualAidTrue, 'no')
print('Do you have issues reading on a smartphone screen?\t\t\t', canReadSmartphoneScreenTrue, 'yes \t\t', n-canReadSmartphoneScreenTrue, 'no')
print('Do you have issues reading cinema subtitles and signs on the subway?\t', canReadCinemaSubtitlesTrue, 'yes \t', n-canReadCinemaSubtitlesTrue, 'no')
print('Do you have issues reading road signs?\t\t\t\t\t', canReadRoadSignsTrue, 'yes \t', n-canReadRoadSignsTrue, 'no')
print('Was the eye test easy for you?\t\t\t\t\t\t', wasEasyTrue, 'yes \t', n-wasEasyTrue, 'no')
print('Do you think you need a (new) visual aid?\t\t\t\t', needsNewVisualAidTrue, 'yes \t', n-needsNewVisualAidTrue, 'no')

In [ ]:
unique_subjects_df.corr().round(2)

# Explorative Data Analysis

In [ ]:
plt_df = pd.DataFrame()
df_list = []
substring = 'mean'
for k in ['jaw', 'eye', 'mouth', 'cheek', 'brow', 'nose']: # 'tongue', 
    filtered_columns = [col for col in df.columns if ((substring in col) & (k in col))] + ['label', 'logMAR', 'suppress_squinting', 'letter_row', 'subject']
    temp = pd.DataFrame(df[filtered_columns].groupby(['label', 'logMAR', 'suppress_squinting', 'letter_row', 'subject']).mean().mean(axis=1)).reset_index()
    temp['feature'] = k
    df_list.append(temp)
    plt_df = pd.concat([plt_df, temp])
plt_df = plt_df.reset_index()
plt_df = plt_df.rename({0: 'value'}, axis=1)

In [ ]:
start_df = df_list[0]
start_df = start_df.rename({0: start_df['feature'][0]}, axis=1).drop('feature', axis=1)
for k in df_list[1:]:
    k = k.rename({0: k['feature'][0]}, axis=1).drop('feature', axis=1)
    start_df = start_df.merge(k, on=['label', 'logMAR', 'suppress_squinting', 'letter_row', 'subject'], how='left')

In [ ]:
for k in plt_df.feature.unique():
    plt.figure()
    sns.histplot(plt_df[plt_df.feature == k]['value']).set_title(k)
    plt.plot()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
hue_order = ['all', 'some', 'none']
sns.boxplot(data=plt_df, x="feature", y="value", hue="label", hue_order=hue_order)
ax.legend(title='Label', loc='lower left')
fig.savefig('../media/boxplot_feature.png', dpi=dpi, bbox_inches='tight')

From the two plots above (histogram and boxplot) we can see that at least two requirements for ANOVA are fulfilled:
1. Samples origina from a normal distribution
2. Standard deviation is equal among the groups

In [ ]:
for k in plt_df.feature.unique():
    filt = plt_df.feature == k
    a = plt_df[filt & (plt_df.label == 'all')]['value']
    b = plt_df[filt & (plt_df.label == 'some')]['value']
    c = plt_df[filt & (plt_df.label == 'none')]['value']
    print(k, '\t', stats.f_oneway(a, b, c))

In [ ]:
for k in plt_df.feature.unique():
    filt = plt_df.feature == k
    rv1 = plt_df[filt & (plt_df.label == 'all')]['value']
    rv2 = plt_df[filt & (plt_df.label == 'none')]['value']
    print(k, stats.ttest_ind(rv1, rv2, equal_var=False))

In [ ]:
g = sns.FacetGrid(plt_df[(plt_df.letter_row > 0) & (plt_df.letter_row < 15)], col="feature", col_wrap=3)
g.map(sns.lineplot, "logMAR", 'value', "suppress_squinting")
g.set_xlabels('logMAR')

border_none = np.mean(plt_df[(plt_df.suppress_squinting == True) & (plt_df.label == 'none')]['logMAR'])
border_some = np.mean(plt_df[(plt_df.suppress_squinting == True) & (plt_df.label == 'some')]['logMAR'])

for ax in g.axes.flat:
    ax.set_xlim(1.1, -0.4)
    line1 = ax.axvline(x=border_none, color='red', linestyle='--', linewidth=1)
    line2 = ax.axvline(x=border_some, color='orange', linestyle='--', linewidth=1)

red_line = mlines.Line2D([], [], color='red', linestyle='--', linewidth=1, label='no letters')
orange_line = mlines.Line2D([], [], color='orange', linestyle='--', linewidth=1, label='some letters')

g.fig.legend(handles=[red_line, orange_line], title="Mean logMAR of the event: Could read", loc='upper right', ncol=3)

handles, labels = g.axes.flat[0].get_legend_handles_labels()
g.fig.legend(handles, labels, title="Suppress squinting", bbox_to_anchor=(0.4, 1), loc='lower center', ncol=3)

g.savefig('../media/change_feature.png', dpi=dpi)
plt.show()

# Machine Learning

In [ ]:
filter_ = (df.letter_row > 0) & (df.letter_row < 15) & (df.suppress_squinting == True)

In [ ]:
drop_cols = [
    'index', 'subject', 'hasVisualAid', 'usesVisualAid', 'needsNewVisualAid',
    'canReadCinemaSubtitles', 'canReadSmartphoneScreen', 'canReadRoadSigns',
    'wasEasy', 'distance', 'visualAidDiopters', 'letter_row',
    'suppress_squinting', 'label', 'label_bin', 'label_mult', 'logMAR', 'fontSizePt',
] + [col for col in df.columns if (('_R' in col))]

In [ ]:
X = df[filter_].drop(drop_cols, axis=1)

y_bin = df[filter_]['label_bin']
y_mult = df[filter_]['label_mult']

X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, random_state=42)

In [ ]:
#best
parameters = {
    'colsample_bytree': [0.2, 0.5, 0.8, 1.0],
    'alpha': [1, 5, 10, 20, 40],
    'lambda': [1, 5, 10, 20, 40],
    'max_depth': [2, 3],
    'n_estimators': [10, 25, 50],
}

In [ ]:
estimator = xgb.XGBClassifier(
    objective='binary:hinge',
    nthread=8,
    seed=42,
)

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 20,
    cv = 20,
    verbose=4
)

In [ ]:
grid_search.fit(X_train, y_train, verbose=0)

[CV 12/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=10;, score=0.500 total time=   0.1s
[CV 19/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=25;, score=0.564 total time=   0.3s
[CV 17/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=50;, score=0.593 total time=   0.4s
[CV 7/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=25;, score=0.764 total time=   0.3s
[CV 2/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=50;, score=0.500 total time=   0.5s
[CV 4/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=10;, score=0.467 total time=   0.1s
[CV 18/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=10;, score=0.700 total time=   0.1s
[CV 20/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=25;, score=0.629 total time=   0.2s
[CV 20/20] END alpha=1, colsample_bytree=0.2, lambda=5, max

[CV 17/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=10;, score=0.564 total time=   0.2s
[CV 13/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=25;, score=0.764 total time=   0.3s
[CV 10/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=50;, score=0.564 total time=   0.4s
[CV 15/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=10;, score=0.800 total time=   0.1s
[CV 15/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=25;, score=0.793 total time=   0.3s
[CV 16/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=50;, score=0.629 total time=   0.5s
[CV 20/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=10;, score=0.629 total time=   0.1s
[CV 18/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=25;, score=0.700 total time=   0.2s
[CV 19/20] END alpha=1, colsample_bytree=0.2, lambda=5, 

[CV 14/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=10;, score=0.529 total time=   0.1s
[CV 18/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=25;, score=0.700 total time=   0.3s
[CV 13/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=50;, score=0.664 total time=   0.4s
[CV 6/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=25;, score=0.764 total time=   0.3s
[CV 8/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=50;, score=0.564 total time=   0.6s
[CV 13/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=10;, score=0.664 total time=   0.1s
[CV 12/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=25;, score=0.700 total time=   0.2s
[CV 11/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=50;, score=0.664 total time=   0.4s
[CV 8/20] END alpha=1, colsample_bytree=0.2, lambda=5, max

[CV 1/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=10;, score=0.625 total time=   0.3s
[CV 6/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=25;, score=0.764 total time=   0.2s
[CV 4/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=50;, score=0.558 total time=   0.4s
[CV 4/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=10;, score=0.433 total time=   0.1s
[CV 18/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=10;, score=0.564 total time=   0.1s
[CV 20/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=25;, score=0.729 total time=   0.3s
[CV 20/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=50;, score=0.793 total time=   0.5s
[CV 6/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=25;, score=0.764 total time=   0.2s
[CV 7/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_de

[CV 7/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=10;, score=0.564 total time=   0.2s
[CV 17/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=25;, score=0.564 total time=   0.3s
[CV 18/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=2, n_estimators=50;, score=0.729 total time=   0.4s
[CV 1/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=25;, score=0.625 total time=   0.3s
[CV 4/20] END alpha=1, colsample_bytree=0.2, lambda=1, max_depth=3, n_estimators=50;, score=0.433 total time=   0.6s
[CV 9/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=10;, score=0.864 total time=   0.1s
[CV 11/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=25;, score=0.564 total time=   0.2s
[CV 13/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_depth=2, n_estimators=50;, score=0.764 total time=   0.4s
[CV 13/20] END alpha=1, colsample_bytree=0.2, lambda=5, max_

...

[CV 2/20] END alpha=10, colsample_bytree=0.5, lambda=1, max_depth=3, n_estimators=50;, score=0.683 total time=   0.7s
[CV 3/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=10;, score=0.808 total time=   0.1s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=10;, score=0.564 total time=   0.1s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=25;, score=0.600 total time=   0.3s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=50;, score=0.564 total time=   0.5s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=10;, score=0.600 total time=   0.2s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=25;, score=0.464 total time=   0.4s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=50;, score=0.600 total time=   0.7s
[CV 19/20] END alpha=10, colsample_bytree=0.5, lam

[CV 12/20] END alpha=10, colsample_bytree=0.5, lambda=1, max_depth=3, n_estimators=25;, score=0.800 total time=   0.4s
[CV 13/20] END alpha=10, colsample_bytree=0.5, lambda=1, max_depth=3, n_estimators=50;, score=0.864 total time=   0.7s
[CV 11/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=10;, score=0.793 total time=   0.1s
[CV 12/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=25;, score=0.800 total time=   0.3s
[CV 8/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=2, n_estimators=50;, score=0.557 total time=   0.5s
[CV 7/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=10;, score=0.829 total time=   0.2s
[CV 6/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=25;, score=0.864 total time=   0.4s
[CV 6/20] END alpha=10, colsample_bytree=0.5, lambda=5, max_depth=3, n_estimators=50;, score=0.864 total time=   0.7s
[CV 10/20] END alpha=10, colsample_bytree=0.5, lambd

[CV 13/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=2, n_estimators=10;, score=0.657 total time=   0.1s
[CV 13/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=2, n_estimators=25;, score=0.729 total time=   0.3s
[CV 13/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=2, n_estimators=50;, score=0.829 total time=   0.6s
[CV 13/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=10;, score=0.793 total time=   0.4s
[CV 14/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=25;, score=0.657 total time=   0.9s
[CV 15/20] END alpha=10, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=50;, score=0.893 total time=   1.6s
[CV 15/20] END alpha=10, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=10;, score=0.893 total time=   0.3s
[CV 15/20] END alpha=10, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=25;, score=0.864 total time=   0.6s
[CV 15/20] END alpha=10, colsample_bytre

[CV 12/20] END alpha=10, colsample_bytree=0.8, lambda=5, max_depth=2, n_estimators=25;, score=0.800 total time=   0.4s
[CV 12/20] END alpha=10, colsample_bytree=0.8, lambda=5, max_depth=2, n_estimators=50;, score=0.800 total time=   0.7s
[CV 12/20] END alpha=10, colsample_bytree=0.8, lambda=5, max_depth=3, n_estimators=10;, score=0.829 total time=   0.5s
[CV 16/20] END alpha=10, colsample_bytree=0.8, lambda=5, max_depth=3, n_estimators=25;, score=0.664 total time=   1.1s
[CV 18/20] END alpha=10, colsample_bytree=0.8, lambda=5, max_depth=3, n_estimators=50;, score=0.600 total time=   2.2s
[CV 3/20] END alpha=10, colsample_bytree=0.8, lambda=10, max_depth=2, n_estimators=25;, score=0.808 total time=   0.8s
[CV 3/20] END alpha=10, colsample_bytree=0.8, lambda=10, max_depth=2, n_estimators=50;, score=0.808 total time=   1.0s
[CV 1/20] END alpha=10, colsample_bytree=0.8, lambda=10, max_depth=3, n_estimators=10;, score=0.625 total time=   0.2s
[CV 1/20] END alpha=10, colsample_bytree=0.8, la

GridSearchCV(cv=20,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=8,
                                     num_parallel_tree=None,
                                     objective='binary:hinge', ...),
             n_jobs=20,
             param_grid={'alpha': [1, 5, 10, 20, 40],
                         'colsample_bytree': [0.2, 0.5, 0.8, 1.0],
                         'lambda': [1, 5, 10, 20, 40], 'max_depth': [2, 3],
                         'n_estimators': [10, 25, 50]},
             scoring='roc_auc', verbose=4)

In [ ]:
print('Best AUC score:', grid_search.best_score_)

In [ ]:
predictions = grid_search.predict(X_train)
accuracy = accuracy_score(y_train, predictions)
print("Accuracy: %.2f%%" % (accuracy*100.0))

In [ ]:
predictions = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy*100.0))

In [ ]:
print('The train set has a size of', len(X_train), 'including', np.sum(y_train), 'positive samples')
print('The test set has a size of', len(X_test), 'including', np.sum(y_test), 'positive samples')

In [ ]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['All', 'Not All'])
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
print('sensitivity', np.round(cm[1,1] / (cm[1,1] + cm[1,0]), 2))
print('specificity', np.round(cm[0,0] / (cm[0,0] + cm[0,1]), 2))

In [ ]:
def rename_columns(old_name):
    name_with_sides = old_name.replace('_L', ' left').replace('_R', ' right')
    match = re.search(r'(.*)(mean|min|max|std)$', name_with_sides)
    if match:
        base_name = match.group(1)
        measure = match.group(2)
        base_parts = re.split('(?=[A-Z_])', base_name)
        base_parts = [part.replace('_', ' ').strip() for part in base_parts if part.strip()]
        new_name = f"{measure} {' '.join(base_parts)}".upper()
        new_name = ' '.join(word.capitalize() for word in new_name.split())
    else:
        new_name = ' '.join(word.capitalize() for word in old_name.split())
    return new_name

In [ ]:
X.columns = [rename_columns(col) for col in X.columns]
explainer = shap.Explainer(grid_search.best_estimator_, X)
shap_values = explainer(X)

In [ ]:
plt.plot()
fig = shap.plots.beeswarm(shap_values, show=False)
plt.savefig('../media/explanation_shap.png', dpi=dpi, bbox_inches='tight')

# Experiment for Accuracy

In [ ]:
def zscore_normalize(x):
    return (x - x.mean()) / x.std()

In [ ]:
df_accuracy = pd.DataFrame()

In [ ]:
for k in glob.glob('../temp_/*.csv'):
    df_accuracy = pd.concat([df_accuracy, pd.read_csv(k)])

In [ ]:
df_accuracy.reset_index(inplace=True)

In [ ]:
df_accuracy = df_accuracy.rename({'index': 'time'}, axis=1)
browUp = df_accuracy[(df_accuracy.letter_row == 3) & (df_accuracy.time > 10)][['browOuterUp_R', 'subject', 'time']]
mouthUp = df_accuracy[(df_accuracy.letter_row == 4) & (df_accuracy.time > 10)][['mouthSmile_R', 'subject', 'time']]
squint = df_accuracy[(df_accuracy.letter_row == 2) & (df_accuracy.time > 10)][['cheekSquint_R', 'subject', 'time']]
browUp = browUp.rename({'browOuterUp_R': 'value'}, axis=1)
mouthUp = mouthUp.rename({'mouthSmile_R': 'value'}, axis=1)
squint = squint.rename({'cheekSquint_R': 'value'}, axis=1)
browUp['action'] = 'brow raising'
mouthUp['action'] = 'mouth smiling'
squint['action'] = 'cheek squinting'

In [ ]:
df_accuracy = pd.concat([browUp, mouthUp, squint])

In [ ]:
df_accuracy.loc[:, 'value'] = df_accuracy.loc[:, ['value', 'subject']].groupby('subject').transform(zscore_normalize)

In [ ]:
df_accuracy['time_shifted'] = df_accuracy.groupby(['subject', 'action'])['time'].transform(lambda x: x - x.min())

g = sns.FacetGrid(df_accuracy, col="subject", row='action', sharex=False)
g.map(sns.lineplot, "time_shifted", "value")
g.set_titles("Subject: {col_name}\nAction: {row_name}")

for ax, title in zip(g.axes[:,0], df_accuracy['action'].unique()):
    ax.set_ylabel(f'Value for {title}')

g.set_xlabels('Time')
g.savefig('../media/predefined_actions.png', dpi=dpi)

[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=50;, score=0.529 total time=   0.6s
[CV 13/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=10;, score=0.629 total time=   0.1s
[CV 13/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.629 total time=   0.2s
[CV 6/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=0.800 total time=   0.5s
[CV 5/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.621 total time=   0.2s
[CV 4/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.367 total time=   0.4s
[CV 8/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=50;, score=0.557 total time=   0.6s
[CV 10/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=10;, score=0.657 total time=   0.2s
[CV 9/20] END alpha=20, colsample_bytree=0.8,

[CV 16/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=10;, score=0.821 total time=   0.1s
[CV 16/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.764 total time=   0.3s
[CV 17/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=1.000 total time=   0.5s
[CV 20/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.721 total time=   0.2s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.700 total time=   0.4s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=50;, score=0.700 total time=   0.6s
[CV 3/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=25;, score=0.808 total time=   0.3s
[CV 6/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=50;, score=0.929 total time=   0.5s
[CV 3/20] END alpha=20, colsample_bytree=0.8

[CV 17/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=25;, score=0.900 total time=   0.5s
[CV 16/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=50;, score=0.900 total time=   0.6s
[CV 15/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=10;, score=0.893 total time=   0.1s
[CV 15/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.900 total time=   0.3s
[CV 15/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=0.900 total time=   0.5s
[CV 18/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.529 total time=   0.2s
[CV 18/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.700 total time=   0.4s
[CV 18/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=50;, score=0.600 total time=   0.6s
[CV 19/20] END alpha=20, colsample_bytre

[CV 11/20] END alpha=20, colsample_bytree=0.5, lambda=10, max_depth=2, n_estimators=25;, score=0.793 total time=   0.3s
[CV 11/20] END alpha=20, colsample_bytree=0.5, lambda=10, max_depth=2, n_estimators=50;, score=0.693 total time=   0.5s
[CV 13/20] END alpha=20, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=10;, score=0.793 total time=   0.2s
[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=25;, score=0.629 total time=   0.4s
[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=10, max_depth=3, n_estimators=50;, score=0.629 total time=   0.6s
[CV 16/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=10;, score=0.821 total time=   0.3s
[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=25;, score=0.529 total time=   0.6s
[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=50;, score=0.529 total time=   0.9s
[CV 15/20] END alpha=20, colsample_bytre

[CV 2/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=50;, score=0.525 total time=   0.6s
[CV 3/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=10;, score=0.592 total time=   0.1s
[CV 1/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.625 total time=   0.3s
[CV 3/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=0.592 total time=   0.4s
[CV 1/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.558 total time=   0.2s
[CV 15/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.857 total time=   0.2s
[CV 14/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.529 total time=   0.4s
[CV 15/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=50;, score=0.864 total time=   0.6s
[CV 4/20] END alpha=20, colsample_bytree=0.8,

[CV 20/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=2, n_estimators=50;, score=0.593 total time=   0.8s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=10;, score=0.593 total time=   0.2s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=25;, score=0.529 total time=   0.5s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=20, max_depth=3, n_estimators=50;, score=0.529 total time=   0.6s
[CV 2/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.492 total time=   0.3s
[CV 2/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=0.525 total time=   0.5s
[CV 8/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=10;, score=0.550 total time=   0.2s
[CV 9/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.729 total time=   0.4s
[CV 6/20] END alpha=20, colsample_bytree=0.5

[CV 20/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=10;, score=0.721 total time=   0.1s
[CV 19/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=25;, score=0.700 total time=   0.3s
[CV 20/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=2, n_estimators=50;, score=0.657 total time=   0.5s
[CV 2/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=25;, score=0.525 total time=   0.4s
[CV 4/20] END alpha=20, colsample_bytree=0.5, lambda=40, max_depth=3, n_estimators=50;, score=0.400 total time=   0.6s
[CV 9/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=10;, score=0.793 total time=   0.2s
[CV 8/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=25;, score=0.621 total time=   0.3s
[CV 2/20] END alpha=20, colsample_bytree=0.8, lambda=1, max_depth=2, n_estimators=50;, score=0.525 total time=   0.5s
[CV 5/20] END alpha=20, colsample_bytree=0.8, la